In [1]:
pip install pandas


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import json 
import os

burned_sensors = []



In [3]:

NUM_SENSORS = '25'
NUM_FIRE = "1"
PATH = "/home/gabrielmadruga/Documents/Github/tcc-wildfire/"

In [3]:
sensors_data = []
for filename in os.listdir(f"{PATH}/simulator/config/qty_sensor_{NUM_SENSORS}/"):
    if filename.endswith(".json"):
      file = open(f"{PATH}/simulator/config/qty_sensor_{NUM_SENSORS}/{filename}")
      json_data = json.load(file)["sensors"]
      sensors_data.extend(json_data)
      

In [107]:
x_limit = sensors_data[0]["pixel_position_x"]
y_limit = sensors_data[0]["pixel_position_y"]
for sensor in sensors_data:
    pixel_position_x = sensor["pixel_position_x"]
    pixel_position_y = sensor["pixel_position_y"]
    sensor["max_x"] = pixel_position_x + x_limit
    sensor["min_x"] = pixel_position_x - x_limit
    sensor["max_y"] = pixel_position_y + y_limit
    sensor["min_y"] = pixel_position_y - y_limit


In [108]:
 
def determinar_regiao(position):
    x, y = eval(position)
    for sensor in sensors_data:
        if (sensor["min_x"] <= x <= sensor["max_x"]) and (sensor["min_y"] <= y <= sensor["max_y"]):
            return sensor["id"]
    return None

In [109]:
for sensor_data in range(1,20):
    data = {}
    df = pd.read_csv(f"{PATH}/simulator/config/fire_data/fire_data_{sensor_data}.csv")
    df["setor"] = df["position"].apply(determinar_regiao)
    df_setor = df.groupby("setor").size().reset_index(name="count")
    data["burned_areas"] = df_setor["setor"].to_list()
    data["fire_data"] = sensor_data
    data["qty_sensor"] = NUM_SENSORS
    burned_sensors.append(data) 

df_burned_areas = pd.DataFrame(burned_sensors)

## Calculate Sensor Response

In [93]:
sensors_data = []

# for filename in os.listdir(f"{PATH}/simulator/config/qty_sensor_{NUM_SENSORS}/data/"):
    # df = pd.read_csv(filename)  

In [98]:
NUM_SENSORS = '25'
NUM_FIRE = "1"
PATH = "/home/gabrielmadruga/Documents/Github/tcc-wildfire/"

for index in range(1,20):
    data = {}
    df = pd.read_csv(f"{PATH}/simulator/config/qty_sensor_{NUM_SENSORS}/data/sensor_data_{index}.csv")
    sensors_detected = []
    # Check Burned Sensors
    burned_sensors = df[df["temperature"] > 55]
    burned_sensors = burned_sensors[burned_sensors["step"]>1]
    burned_sensors = burned_sensors.groupby("id").size().reset_index(name="count")["id"].to_list()

    # Check Detected Sensors
    fire_detected_sensors = df[(df["step"] - 1)%5 ==0]
    fire_detected_sensors["temperature_over"] = fire_detected_sensors["temperature"].apply(lambda x: 1 if x > 30 else 0)
    fire_detected_sensors["humidity_under"] = fire_detected_sensors["humidity"].apply(lambda x: 1 if x < 30 else 0)
    fire_detected_sensors = fire_detected_sensors.groupby("id").agg({"temperature_over":"sum","humidity_under":"sum"})

    # Check Temperature Detection
    temperature_detected_sensors = fire_detected_sensors[fire_detected_sensors["temperature_over"] > 1]
    temperature_detected_sensors = temperature_detected_sensors["temperature_over"].index.to_list()

    # Check Humidity Detection 
    humidity_detected_sensors = fire_detected_sensors[fire_detected_sensors["humidity_under"] > 1]
    humidity_detected_sensors = humidity_detected_sensors["humidity_under"].index.to_list()

    #Join Data
    sensors_detected.extend(burned_sensors)
    sensors_detected.extend(temperature_detected_sensors)
    sensors_detected.extend(humidity_detected_sensors)
    sensors_detected = list(dict.fromkeys(sensors_detected))
    data["fire_data"] = index
    data["qty_sensors"] = NUM_SENSORS
    data["detected_sensors"] = sensors_detected
    sensors_data.append(data)

len(sensors_data)    



/tmp/ipykernel_6273/2380364336.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_detected_sensors["temperature_over"] = fire_detected_sensors["temperature"].apply(lambda x: 1 if x > 30 else 0)
/tmp/ipykernel_6273/2380364336.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fire_detected_sensors["humidity_under"] = fire_detected_sensors["humidity"].apply(lambda x: 1 if x < 30 else 0)
/tmp/ipykernel_6273/2380364336.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

95

In [90]:
len(sensors_data)

90

In [99]:
df = pd.DataFrame(sensors_data)

In [100]:
df

,fire_data,qty_sensors,detected_sensors
0,1,01,[]
1,2,01,[1]
2,3,01,[1]
3,4,01,[1]
4,5,01,[1]
...,...,...,...
90,15,25,"[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
91,16,25,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
92,17,25,"[1, 2, 6, 7, 11, 12, 13, 16, 17, 18, 21, 22, 2..."
93,18,25,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [101]:
df.to_csv("detected_areas.csv")

## DataFrame Compare

In [117]:
burned_areas = pd.read_csv("burned_areas.csv")
detected_areas = pd.read_csv("detected_areas.csv")
compare_df = pd.merge(burned_areas,detected_areas,how="inner",on=["fire_data","qty_sensor"])
compare_df

,Unnamed: 0_x,burned_areas,fire_data,qty_sensor,Unnamed: 0_y,detected_sensors
0,0,[1],1,1,0,[]
1,1,[1],2,1,1,[1]
2,2,[1],3,1,2,[1]
3,3,[1],4,1,3,[1]
4,4,[1],5,1,4,[1]
...,...,...,...,...,...,...
90,90,"[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",15,25,90,"[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
91,91,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",16,25,91,"[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
92,92,"[1, 2, 6, 7, 11, 12, 13, 16, 17, 18, 19, 21, 2...",17,25,92,"[1, 2, 6, 7, 11, 12, 13, 16, 17, 18, 21, 22, 2..."
93,93,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",18,25,93,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [114]:
def check_vp(burned_areas,detected_sensors,qty_sensor):
    total = 0
    for sensor in range(1,qty_sensor):
        if sensor in burned_areas:
            if sensor in detected_sensors:
                total = total + 1
    return total            

def check_fp(burned_areas,detected_areas):
    pass

def check_vn(burned_areas,detected_areas):
    return 

def check_fn(burned_areas,detected_areas):
    pass

In [116]:
compare_df['VP'] = compare_df.apply(lambda row:check_vp(row["burned_areas"],row["detected_sensors"],row["qty_sensor"]),axis=1)

KeyError: 'detected_areas'